<a href="https://colab.research.google.com/github/LYZAcell/data_study/blob/main/%EB%B3%B5%EC%8A%B5_%EC%9D%B4%EA%B0%80%EC%98%8141_0312.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
# 용량이 크니 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
df = pd.read_csv("./drive/MyDrive/BDA(ML)/수)BDA_추천알고리즘/과제/dataset/2019-Dec.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3533286 entries, 0 to 3533285
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 242.6+ MB


In [ ]:
# na 여부
df.isna().sum()

,0
event_time,0
event_type,0
product_id,0
category_id,0
category_code,3474821
brand,1510289
price,0
user_id,0
user_session,779


In [ ]:
## 데이터 샘플로 뽑아서 분석 진행 _ 5만개, 랜덤씨드는 111
df_sample =df.sample(n=50000, random_state=111)
print(df_sample)

                      event_time        event_type  product_id  \
1141402  2019-12-09 19:04:29 UTC              view     5814516   
2826120  2019-12-23 12:44:51 UTC              view     5915004   
354426   2019-12-03 16:25:54 UTC              view     5877456   
3089023  2019-12-25 19:06:15 UTC              cart     5608703   
1762400  2019-12-14 11:41:25 UTC              view     5769914   
...                          ...               ...         ...   
954138   2019-12-08 14:29:44 UTC  remove_from_cart     5695961   
1363136  2019-12-11 10:17:56 UTC              cart     5800038   
3033914  2019-12-25 10:03:55 UTC          purchase     5629984   
3455329  2019-12-30 08:41:10 UTC          purchase     5851607   
2220662  2019-12-17 21:18:58 UTC              cart     5678044   

                 category_id category_code     brand   price    user_id  \
1141402  1487580013539623112           NaN       NaN   13.49  247790217   
2826120  1487580008028308447           NaN  babyliss   96

# 퍼널이란
- 마케팅, 영업과정에서 고객의 구매여정을 시각적으로 나타낸 모델
- 고객이 제품, 서비스를 인식하고 구매에 이르는 과정을 말함

1. 인지 -> 2. 관심 -> 3. 욕구 -> 4. 행동(구매)

- 위쪽이 넓고 아래쪽이 좁은 모양 => 잠재 고객이 인지에는 흔하지만 진행될수록 이탈하는 고객이 많아지는 것을 반영


- 퍼널이 닫혀있다 = 구매과정이 더이상 진행되지 않는 상황 (잠재 고객이 실재 고객으로 이어지지 않는 경우)

In [ ]:
# 퍼널이 닫혀있다 가정하면, event가 무엇이 있는지 확인
df_sample.event_type.value_counts()

,count
event_type,
view,24451
cart,13178
remove_from_cart,9310
purchase,3061


In [ ]:
# 퍼널 비교 시 이벤트 별 제품이 다른지, 다량/단품 구매 시 범위
# 어떤 제품끼리 묶여 구매갇 ㅚ는지, product_id 추천 및 커버 범위

# 추천 구현 시 개인화 추천시스템을 단순/복잡 로직이 있음.
# 가장 많이 구매한 것, 가장 같이 잘 팔리는 것(co-occurance 동시구매), 이벤트 별 추천, 제품 또는 검색어 기반으로 추천

# 가설1. Event별 제품의 베스트가 차이가 클까?
top_viewed=df_sample[df_sample['event_type']=='view']['product_id'].value_counts().head(5)
top_carted=df_sample[df_sample['event_type']=='cart']['product_id'].value_counts().head(5)
top_purchased=df_sample[df_sample['event_type']=='purchase']['product_id'].value_counts().head(5)

In [ ]:
print('top_viewed',top_viewed)
print("\n\n")
print('top_carted',top_carted)
print("\n\n")
print('top_purchased',top_purchased)

top_viewed product_id
5809910    353
5909810    115
5877454     94
5809912     89
5886282     81
Name: count, dtype: int64



top_carted product_id
5809910    90
5700037    80
5802432    75
5854897    52
5815662    47
Name: count, dtype: int64



top_purchased product_id
5809910    22
5700037    12
5854897    11
5528035    11
5802432    11
Name: count, dtype: int64


## 코호트 컬럼 정리(Cohort Column Analysis)
- 특정 그룹 사용자나 고객을 시간에 따라 분석, 비교하는 방법으로 주로 마케팅, 제품관리, 사용자 경험분석에 사용
- 코호트: 특정한 특성이나 행동을 공유하는 사용자 집단을 의미함(Ex: 특정 기간에 가입한 사용자들, 특정 캠페인을 통해 유입된 고객들)


```
daily_funnel_cohort=daily_funnel_cohort[['view','cart','purchase']]
```
로 daily_funnel_cohort에서 view, acart, purchase추출해서 cohort용 컬럼 추출 (컬럼 정리)

In [ ]:
## 퍼널에 대한 확인

# 시계열로 변환
df_sample['event_time'] = pd.to_datetime(df_sample['event_time'])

## event_time 일자로 뽑으려면
df_sample['date']=df_sample['event_time'].dt.date

## 우리가 지금 퍼널을 카운팅하기 하기 위해서는 어떤 기준으로 가야할까?
## user_id 카운팅
daily_funnel_cohort=df_sample.groupby(['date','event_type'])['user_id'].nunique().unstack(fill_value=0)

## cohort 컬럼 정리
daily_funnel_cohort=daily_funnel_cohort[['view','cart','purchase']]

## 전환율 계산
daily_funnel_cohort['view->cart(%)'] = (daily_funnel_cohort['cart']/daily_funnel_cohort['view']*100).round(2)
daily_funnel_cohort['cart->purchase(%)'] = (daily_funnel_cohort['purchase']/daily_funnel_cohort['cart']*100).round(2)
daily_funnel_cohort['view->purchase(%)'] = (daily_funnel_cohort['purchase']/daily_funnel_cohort['view']*100).round(2)

In [ ]:
# 코호트 퍼널
daily_funnel_cohort

event_type,view,cart,purchase,view->cart(%),cart->purchase(%),view->purchase(%)
date,,,,,,
2019-12-01,766,401,89,52.35,22.19,11.62
2019-12-02,825,413,89,50.06,21.55,10.79
2019-12-03,805,460,115,57.14,25.00,14.29
2019-12-04,779,439,103,56.35,23.46,13.22
2019-12-05,766,393,122,51.31,31.04,15.93
2019-12-06,719,404,105,56.19,25.99,14.60
2019-12-07,696,381,80,54.74,21.00,11.49
2019-12-08,753,403,83,53.52,20.60,11.02
2019-12-09,936,484,102,51.71,21.07,10.90
